<h1>Capstone Project #1</h1>
<h2>Predicting Customer Churn</h2>
<h3>Cliff Robbins</h3>

<h3>Proposal</h3>
<p>My project will focus on a problem that 28 million business face each day of operation, customer churn.</p>

<h3>Description:</h3>
<p><strong>Customer churn</strong>, also known as customer attrition, customer turnover or customer defection is the loss of clients or customers.  Many companies include customer churn rate as part of their monitoring metrics because the cost of retaining current customers compared to acquiring new customers is much less.  
Within customer churn there is the concept of voluntary and involuntary churn with voluntary being a customer leaves on their own choice while involuntary could be attributed to customer relocation to a long term care facility, death or customer relocation in a different state/geography.  In most analytical models, involuntary churn is excluded from the metric.
</p>

<h3>Formulation of a Question</h3>
<p>When a company first starts up, the founding members can typically handle all of the various customer concerns.  As the company continues to grow, the founders can no longer service all of the various clients with support handled by a customer service team.  The customer service team focuses on current issues and a proactive approach is lost.</p>
<p>As the company grows, the company still cares about its clients; however, due to the large customer base they can no longer address each and every customer.  This is a real problem for companies.  How does a company proactively predict if a customer is happy or unhappy?  How does a company know if a customer is so unhappy that they are willing to leave?  If a company knew if a customer was getting ready to leave, could they reach out to the customer and mend the relationship?</p>
<h3>Hypothesis</h3>
<p>I believe past customer data can predict future customer churn. </p>
<h3>Prediction</h3>
<p>If I had past customer data that showed various features and whether they stayed or churned we could use that data to predict future outcomes of current customers.</p>
<h3>Testing</h3>
<p>To test my hypothesis, I will use a set of customer data with various features along with whether they churned or not.</p>
<p>The data has 7043 rows and can be found at:</p>
<p>https://www.kaggle.com/blastchar/telco-customer-churn</p>


<h2>Data Cleanup</h2>
<p>I will do various steps to ensure the data is clean and ready to use for prediction.</p>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import numpy as np
import h5py

/Users/cliff/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#the data location
customer_churn_file = "data/WA_Fn-UseC_-Telco-Customer-Churn.csv"

#load the data
raw_customer_churn_df = pd.read_csv(customer_churn_file,index_col=0,header=0)

#display first 5 rows
raw_customer_churn_df.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
#lets check for null or missing values

df_null = raw_customer_churn_df[raw_customer_churn_df.isnull().any(axis=1)]

print('Check for null values.')
print(df_null)
print('---------------------------------------')
print()
print('Find empty strings.')
print(np.where(raw_customer_churn_df.applymap(lambda x: x== ' ')))
print('---------------------------------------')
print()
raw_customer_churn_df.info()

Check for null values.
Empty DataFrame
Columns: [gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges, Churn]
Index: []
---------------------------------------

Find empty strings.
(array([ 488,  753,  936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754]), array([18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]))
---------------------------------------

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 20 columns):
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null objec

In [4]:
#There are some empty strings we need to deal with
empty_strings_df = raw_customer_churn_df[raw_customer_churn_df.TotalCharges == ' ']
print('Number of empty rows: ',len(empty_strings_df))
print('Total number or rows:', len(raw_customer_churn_df))
#Since nothing is null, lets look for unique categories
print('---------------------------------------')
print()
print('gender: ',raw_customer_churn_df.gender.unique().tolist())
print('SeniorCitizen: ',raw_customer_churn_df.SeniorCitizen.unique().tolist())
print('Partner: ',raw_customer_churn_df.Partner.unique().tolist())
print('Dependents: ',raw_customer_churn_df.Dependents.unique().tolist())
print('tenure: ',raw_customer_churn_df.tenure.unique().tolist())
print('PhoneService: ',raw_customer_churn_df.PhoneService.unique().tolist())
print('MultipleLines: ',raw_customer_churn_df.MultipleLines.unique().tolist())
print('InternetService: ',raw_customer_churn_df.InternetService.unique().tolist())
print('OnlineSecurity: ',raw_customer_churn_df.OnlineSecurity.unique().tolist())
print('OnlineBackup: ',raw_customer_churn_df.OnlineBackup.unique().tolist())
print('DeviceProtection: ',raw_customer_churn_df.DeviceProtection.unique().tolist())
print('TechSupport: ',raw_customer_churn_df.TechSupport.unique().tolist())
print('StreamingTV: ',raw_customer_churn_df.StreamingTV.unique().tolist())
print('StreamingMovies: ',raw_customer_churn_df.StreamingMovies.unique().tolist())
print('Contract: ',raw_customer_churn_df.Contract.unique().tolist())
print('PaperlessBilling: ',raw_customer_churn_df.PaperlessBilling.unique().tolist())
print('PaymentMethod: ',raw_customer_churn_df.PaymentMethod.unique().tolist())


Number of empty rows:  11
Total number or rows: 7043
---------------------------------------

gender:  ['Female', 'Male']
SeniorCitizen:  [0, 1]
Partner:  ['Yes', 'No']
Dependents:  ['No', 'Yes']
tenure:  [1, 34, 2, 45, 8, 22, 10, 28, 62, 13, 16, 58, 49, 25, 69, 52, 71, 21, 12, 30, 47, 72, 17, 27, 5, 46, 11, 70, 63, 43, 15, 60, 18, 66, 9, 3, 31, 50, 64, 56, 7, 42, 35, 48, 29, 65, 38, 68, 32, 55, 37, 36, 41, 6, 4, 33, 67, 23, 57, 61, 14, 20, 53, 40, 59, 24, 44, 19, 54, 51, 26, 0, 39]
PhoneService:  ['No', 'Yes']
MultipleLines:  ['No phone service', 'No', 'Yes']
InternetService:  ['DSL', 'Fiber optic', 'No']
OnlineSecurity:  ['No', 'Yes', 'No internet service']
OnlineBackup:  ['Yes', 'No', 'No internet service']
DeviceProtection:  ['No', 'Yes', 'No internet service']
TechSupport:  ['No', 'Yes', 'No internet service']
StreamingTV:  ['No', 'Yes', 'No internet service']
StreamingMovies:  ['No', 'Yes', 'No internet service']
Contract:  ['Month-to-month', 'One year', 'Two year']
PaperlessBill

In [5]:
#lets update the rows with empty strings to 0.0
#the best way to do this, would be to consult with the client to understand why the data is empty

print(raw_customer_churn_df.loc[raw_customer_churn_df.TotalCharges == '0','TotalCharges'])

raw_customer_churn_df.loc[raw_customer_churn_df.TotalCharges == ' ','TotalCharges'] = 0.0  #we should use NaN instead of 0.0

#NOTE - Leave the data and then take a look to see how this impacts the overall strategy
#STORY TELLING - Need to share this as part of my questions and answers

print('Number of empty rows: ',len(raw_customer_churn_df[raw_customer_churn_df.TotalCharges == ' ']))
print()
print('Total number of rows:',len(raw_customer_churn_df))


Series([], Name: TotalCharges, dtype: object)
Number of empty rows:  0

Total number of rows: 7043


In [6]:
#based on what we are seeing, all of them except tenure can be set to categories
column_data_type_assignments = {'gender':'category','SeniorCitizen':'category','Partner':'category','Dependents':'category','PhoneService':'category','MultipleLines':'category','InternetService':'category','OnlineSecurity':'category','OnlineBackup':'category','DeviceProtection':'category','TechSupport':'category','StreamingTV':'category','StreamingMovies':'category','Contract':'category','PaperlessBilling':'category','PaymentMethod':'category','TotalCharges':'float64','Churn':'category'}
assigned_customer_churn_df = raw_customer_churn_df.astype(column_data_type_assignments,copy=True)

assigned_customer_churn_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 20 columns):
gender              7043 non-null category
SeniorCitizen       7043 non-null category
Partner             7043 non-null category
Dependents          7043 non-null category
tenure              7043 non-null int64
PhoneService        7043 non-null category
MultipleLines       7043 non-null category
InternetService     7043 non-null category
OnlineSecurity      7043 non-null category
OnlineBackup        7043 non-null category
DeviceProtection    7043 non-null category
TechSupport         7043 non-null category
StreamingTV         7043 non-null category
StreamingMovies     7043 non-null category
Contract            7043 non-null category
PaperlessBilling    7043 non-null category
PaymentMethod       7043 non-null category
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null float64
Churn               7043 non-null category
dtypes: category(17), float6

In [7]:
#now lets see if we have any outliers
assigned_customer_churn_df.describe()

,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,2279.734304
std,24.559481,30.090047,2266.794470
min,0.000000,18.250000,0.000000
25%,9.000000,35.500000,398.550000
50%,29.000000,70.350000,1394.550000
75%,55.000000,89.850000,3786.600000
max,72.000000,118.750000,8684.800000


<h1>Findings</h1>
<h2>Goal:</h2>
<p>The goal for data wrangling was to import the data and then clean the data before processing the information.  Below are the various steps I took to investigate the data and then clean the data.</p>
<h2>Investigation:</h2>
<p>The first step was to import the data and the investigate the data.</p>

<p>My data is located in a csv file which I imported into a Panda’s DataFrame using the read_csv function.  I have the data stored in a subfolder under the Jupyter notebook so others can leverage the same data set.</p>

<p>After importing the data, I ran a head function to show the first 5 rows to understand what the data looked like.</p>

<p>I then started looking for missing values. </p>
<p>I started initially looking for any null values by column.  My dataframe came back with zero null values.</p>
<p>I then looked for any empty strings by row.  My results returned 11 rows that had empty strings.</p>
<h2>Cleaning:</h2>
<p>Once I understand what columns had issues, I also wanted to understand if Pandas had assigned the correct types to each column.  I ran a .info method and it showed almost all columns were set to object.  This meant I needed to get a better understanding of each column data type.</p>

<p>Based on the head method, I then listed out each column that I felt was categorical using the unique method and converting them to a list to see the unique values.  Here is the printout:</p>

<p>gender:  ['Female', 'Male']</p>
<p>SeniorCitizen:  [0, 1]</p>
<p>Partner:  ['Yes', 'No']</p>
<p>Dependents:  ['No', 'Yes']</p>
<p>tenure:  [1, 34, 2, 45, 8, 22, 10, 28, 62, 13, 16, 58, 49, 25, 69, 52, 71, 21, 12, 30, 47, 72, 17, 27, 5, 46, 11, 70, 63, 43, 15, 60, 18, 66, 9, 3, 31, 50, 64, 56, 7, 42, 35, 48, 29, 65, 38, 68, 32, 55, 37, 36, 41, 6, 4, 33, 67, 23, 57, 61, 14, 20, 53, 40, 59, 24, 44, 19, 54, 51, 26, 0, 39]</p>
<p>PhoneService:  ['No', 'Yes']</p>
<p>MultipleLines:  ['No phone service', 'No', 'Yes']</p>
<p>InternetService:  ['DSL', 'Fiber optic', 'No']</p>
<p>OnlineSecurity:  ['No', 'Yes', 'No internet service']</p>
<p>OnlineBackup:  ['Yes', 'No', 'No internet service']</p>
<p>DeviceProtection:  ['No', 'Yes', 'No internet service']</p>
<p>TechSupport:  ['No', 'Yes', 'No internet service']</p>
<p>StreamingTV:  ['No', 'Yes', 'No internet service']</p>
<p>StreamingMovies:  ['No', 'Yes', 'No internet service']</p>
<p>Contract:  ['Month-to-month', 'One year', 'Two year']</p>
<p>PaperlessBilling:  ['Yes', 'No']</p>
<p>PaymentMethod:  ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)']</p>

<p>Based on this, I decided that all of them except tenure would be set to a type of category.</p>

<p>I had also noticed that TotalCharges was an object and not a float64, which made me suspicious that something wasn’t right.  When I investigated, it had 11 rows with empty strings.  I looked at the 11 rows and could see that they data was ‘off’.</p>
<h2>Dealing with Missing Values:</h2>
<p>The only column that has missing values was the TotalCharges column.  After looking at the 11 rows, the data looked invalid so I decided to fill in the 11 rows.  I filled in the 11 rows with zero’s and then assigned the column as type float64.</p>
<h2>Outliers:</h2>
<p>After dealing with missing values and assigning the proper types, I then used the describe method so I could take a look at the numerical types and understand if I had any values that looked odd.  Based on that readout, the values appear to me normal of what I would expect for monthly and total charges.</p>


In [10]:
#lets export the data to be used in later notebooks
#write the data
assigned_customer_churn_df.to_hdf('data/cp1-1-assigned-data.hdf5',key='cleaned_data',mode='w',format='table',data_columns=True)
